In [1]:
%pip install -r ../requirements.txt


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt 
import os
import numpy as np
import shap


In [5]:
df = pd.read_csv("../data/Teldata2023_AI_v2.csv")
df.head()

,Unnamed: 0,locatie,datum,startuur,kwartier,N_samenstelling,missing_prev,missing_next,missing_AV,feestdag_nationaal,...,medium,zwaar,alle_voertuigen,I1,I2,I3,I4,I5,I123,I45
0,1,25006H,2023-04-04,0,45,1,0,0,0,0,...,0,1,3,0,11,0,0,9,11,9
1,2,25006T,2023-04-04,0,30,1,0,0,0,0,...,0,0,1,0,11,0,0,9,11,9
2,3,25006H,2023-04-04,0,15,1,0,0,0,0,...,0,0,3,0,11,0,0,9,11,9
3,4,25006H,2023-04-04,0,30,1,0,0,0,0,...,0,0,5,0,11,0,0,9,11,9
4,5,25006T,2023-04-04,0,45,1,0,0,0,0,...,0,1,3,0,11,0,0,9,11,9


In [8]:
df = df.drop("Unnamed: 0",axis=1)


KeyError: "['Unnamed: 0'] not found in axis"

In [12]:
df["uur"] = df["startuur"]*60 + df["kwartier"]
temp = df[["locatie","datum","uur","alle_voertuigen"]]
temp

,locatie,datum,uur,alle_voertuigen
0,25006H,2023-04-04,45,3
1,25006T,2023-04-04,30,1
2,25006H,2023-04-04,15,3
3,25006H,2023-04-04,30,5
4,25006T,2023-04-04,45,3
...,...,...,...,...
4267839,00030T,2023-11-28,540,74
4267840,00030H,2023-11-28,540,142
4267841,00030T,2023-11-28,570,78
4267842,00030H,2023-11-28,570,108


In [13]:
df_wide = temp.pivot_table(index= ["locatie","datum"],columns= 'uur',values='alle_voertuigen')

In [15]:
df_wide.reset_index()

uur,locatie,datum,0,15,30,45,60,75,90,105,...,1290,1305,1320,1335,1350,1365,1380,1395,1410,1425
0,00001H,2023-04-06,2.0,8.0,7.0,6.0,2.0,4.0,6.0,5.0,...,40.0,50.0,42.0,32.0,20.0,15.0,18.0,16.0,15.0,11.0
1,00001H,2023-04-07,7.0,7.0,4.0,8.0,4.0,9.0,3.0,6.0,...,38.0,40.0,30.0,31.0,25.0,24.0,24.0,16.0,18.0,19.0
2,00001H,2023-04-08,12.0,7.0,4.0,7.0,7.0,4.0,5.0,3.0,...,36.0,27.0,26.0,24.0,21.0,27.0,14.0,27.0,15.0,14.0
3,00001H,2023-04-09,10.0,13.0,10.0,13.0,2.0,7.0,3.0,4.0,...,37.0,41.0,23.0,20.0,17.0,14.0,15.0,14.0,13.0,2.0
4,00001H,2023-04-10,7.0,4.0,10.0,6.0,5.0,NaN,2.0,4.0,...,38.0,36.0,28.0,11.0,12.0,8.0,15.0,8.0,7.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46148,33009H,2023-05-20,8.0,9.0,4.0,8.0,3.0,6.0,3.0,4.0,...,16.0,15.0,12.0,15.0,16.0,18.0,15.0,16.0,12.0,10.0
46149,33009H,2023-05-21,12.0,10.0,8.0,8.0,6.0,8.0,4.0,3.0,...,17.0,15.0,13.0,9.0,10.0,8.0,7.0,7.0,6.0,5.0
46150,33009H,2023-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,11.0,24.0,20.0,10.0,6.0,3.0,9.0,7.0,2.0
46151,33009H,2023-05-23,5.0,5.0,4.0,3.0,3.0,2.0,3.0,1.0,...,15.0,13.0,15.0,13.0,10.0,5.0,7.0,6.0,5.0,4.0


In [16]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
imputer.fit_transform(df_wide)


array([[ 2. ,  8. ,  7. , ..., 16. , 15. , 11. ],
       [ 7. ,  7. ,  4. , ..., 16. , 18. , 19. ],
       [12. ,  7. ,  4. , ..., 27. , 15. , 14. ],
       ...,
       [ 4.5,  2.5,  2.5, ...,  9. ,  7. ,  2. ],
       [ 5. ,  5. ,  4. , ...,  6. ,  5. ,  4. ],
       [ 6. ,  5. ,  4. , ...,  6. ,  9. ,  4. ]])

In [19]:
result = imputer.transform(df_wide)

In [20]:
ImputedDF = pd.DataFrame(result)

In [28]:
df_wide.reset_index()

uur,locatie,datum,0,15,30,45,60,75,90,105,...,1290,1305,1320,1335,1350,1365,1380,1395,1410,1425
0,00001H,2023-04-06,2.0,8.0,7.0,6.0,2.0,4.0,6.0,5.0,...,40.0,50.0,42.0,32.0,20.0,15.0,18.0,16.0,15.0,11.0
1,00001H,2023-04-07,7.0,7.0,4.0,8.0,4.0,9.0,3.0,6.0,...,38.0,40.0,30.0,31.0,25.0,24.0,24.0,16.0,18.0,19.0
2,00001H,2023-04-08,12.0,7.0,4.0,7.0,7.0,4.0,5.0,3.0,...,36.0,27.0,26.0,24.0,21.0,27.0,14.0,27.0,15.0,14.0
3,00001H,2023-04-09,10.0,13.0,10.0,13.0,2.0,7.0,3.0,4.0,...,37.0,41.0,23.0,20.0,17.0,14.0,15.0,14.0,13.0,2.0
4,00001H,2023-04-10,7.0,4.0,10.0,6.0,5.0,NaN,2.0,4.0,...,38.0,36.0,28.0,11.0,12.0,8.0,15.0,8.0,7.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46148,33009H,2023-05-20,8.0,9.0,4.0,8.0,3.0,6.0,3.0,4.0,...,16.0,15.0,12.0,15.0,16.0,18.0,15.0,16.0,12.0,10.0
46149,33009H,2023-05-21,12.0,10.0,8.0,8.0,6.0,8.0,4.0,3.0,...,17.0,15.0,13.0,9.0,10.0,8.0,7.0,7.0,6.0,5.0
46150,33009H,2023-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,11.0,24.0,20.0,10.0,6.0,3.0,9.0,7.0,2.0
46151,33009H,2023-05-23,5.0,5.0,4.0,3.0,3.0,2.0,3.0,1.0,...,15.0,13.0,15.0,13.0,10.0,5.0,7.0,6.0,5.0,4.0


In [ ]:
nonImputedDF = df_wide.reset_index()

In [26]:
ImputedDF

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,2.0,8.0,7.0,6.0,2.0,4.0,6.0,5.0,7.00,5.0,...,40.0,50.0,42.0,32.0,20.0,15.0,18.0,16.0,15.0,11.0
1,7.0,7.0,4.0,8.0,4.0,9.0,3.0,6.0,4.00,6.0,...,38.0,40.0,30.0,31.0,25.0,24.0,24.0,16.0,18.0,19.0
2,12.0,7.0,4.0,7.0,7.0,4.0,5.0,3.0,2.00,7.0,...,36.0,27.0,26.0,24.0,21.0,27.0,14.0,27.0,15.0,14.0
3,10.0,13.0,10.0,13.0,2.0,7.0,3.0,4.0,1.00,4.0,...,37.0,41.0,23.0,20.0,17.0,14.0,15.0,14.0,13.0,2.0
4,7.0,4.0,10.0,6.0,5.0,2.5,2.0,4.0,1.75,7.0,...,38.0,36.0,28.0,11.0,12.0,8.0,15.0,8.0,7.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46148,8.0,9.0,4.0,8.0,3.0,6.0,3.0,4.0,3.00,4.0,...,16.0,15.0,12.0,15.0,16.0,18.0,15.0,16.0,12.0,10.0
46149,12.0,10.0,8.0,8.0,6.0,8.0,4.0,3.0,2.00,5.0,...,17.0,15.0,13.0,9.0,10.0,8.0,7.0,7.0,6.0,5.0
46150,4.5,2.5,2.5,3.0,3.5,2.5,2.5,1.0,3.00,2.5,...,15.0,11.0,24.0,20.0,10.0,6.0,3.0,9.0,7.0,2.0
46151,5.0,5.0,4.0,3.0,3.0,2.0,3.0,1.0,1.75,1.5,...,15.0,13.0,15.0,13.0,10.0,5.0,7.0,6.0,5.0,4.0


In [25]:
ImputedDF[ImputedDF["locatie"]== '33009H' & ImputedDF["datum"]== '2023-05-22' ]

KeyError: 'locatie'